In [ ]:
%matplotlib inline

In [ ]:
import h5py
import array
import time

_LMT_TIMESTEP = 5

f = h5py.File( '/global/project/projectdirs/pma/www/daily/2016-06-04/edison_snx11025.h5lmt', 'r')

## Find the time corresponding to each column

In [ ]:
for x in f['FSStepsGroup/FSStepsDataSet'][0:3]:
    print time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(x))

## Find peak metadata op rate

For each metadata operation we track, print out the highest op rate (ops/sec) observed.  To convert this op rate into an op count, multiply by `_LMT_TIMESTEP`.

In [ ]:
for i in range(len(f['MDSOpsGroup/MDSOpsDataSet'].attrs['OpNames'])):
    op_name = f['MDSOpsGroup/MDSOpsDataSet'].attrs['OpNames'][i]
    max_op_rate = f['MDSOpsGroup/MDSOpsDataSet'][i,:].max()
    if max_op_rate > 0.0:
        print op_name, max_op_rate
          

## Find the bytes read by one OST over the whole day

Note that we multiply by `_LMT_TIMESTEP` to get total bytes read.

In [ ]:
ost_target = 'snx11025-OST005c'
ost_target_idx = list(f['OSTReadGroup/OSTBulkReadDataSet'].attrs[u'OSTNames']).index(ost_target)
print "Bytes read: ", f['OSTReadGroup/OSTBulkReadDataSet'][ost_target_idx,:].sum() * _LMT_TIMESTEP

## Find the fraction of data points that are marked as missing

In [ ]:
tot_missing = f['FSMissingGroup/FSMissingDataSet'][:].sum()
# The following multiplies all of the dimensions of the given dataset together
# to get the total number of elements contained
tot_data = reduce(lambda a, b: a*b, f['FSMissingGroup/FSMissingDataSet'].shape)
print "Total data points missing:", tot_missing, "out of", tot_data
print "Fraction of data missing:", float(tot_missing) / tot_data

## Plot the observed bandwidth across the file system for the entire HDF5 file

In [ ]:
# Build the array that we will plot
bytes_per_sec = array.array( 
    'd', 
    [ f['OSTReadGroup/OSTBulkReadDataSet'][:,x].sum() for x in range(f['OSTReadGroup/OSTBulkReadDataSet'].shape[1]) ] )

In [ ]:
from matplotlib import pyplot as plt
plt.xlabel('Time')
plt.ylabel('GiB Read')
plt.title('Read activity')
plt.plot([ x0 * _LMT_TIMESTEP / 3600.0 for x0 in range(len(bytes_per_sec)) ], 
         [ x / 2.0**30 for x in bytes_per_sec ])
